This file will run an LSTM model to predict the electricity prices. 

In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from functions import plot_comparison, evaluate_lstm

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [7]:
df = pd.read_csv('../../Data/zra_sgp_dam.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

# Defining Functions

In [8]:
def encode_cyclic(df, col, max_val):
    """"
    Time features like Hour, Month, day_of_week are cyclical, not linear. 
    Without encoding them properly, the model will misunderstand their relationships.
    """
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / max_val)
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / max_val)
    return df

def preprocess(df):
    df = df.copy()
    
    # Cyclic encode time features
    df = encode_cyclic(df, 'Hour', 24)
    df = encode_cyclic(df, 'Month', 12)
    df = encode_cyclic(df, 'day_of_week', 7)
    
    # Drop unused or problematic columns
    df = df.drop(columns=['Hour', 'Month', 'day_of_week'])  # Keep cyclic versions instead
    
    # Fill/clean if needed
    df = df.fillna(method='ffill').dropna()
    
    return df

In [9]:
# Create sequence for LSTM
def create_sequences(data, target_col, n_steps=48, forecast_horizon=24):
    X, y = [], []
    for i in range(len(data) - n_steps - forecast_horizon):
        seq_x = data.iloc[i:i+n_steps].drop(columns=[target_col]).values
        seq_y = data.iloc[i+n_steps:i+n_steps+forecast_horizon][target_col].values
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [10]:
# Dataset and DataLoader
class PriceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [11]:
# Lstm model
class LSTMForecast(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.2, output_size=24):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout if num_layers > 1 else 0.0, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, _ = self.lstm(x) # Get all outputs
        out = output[:, -1, :] # Use output from the last timestep
        return self.fc(out)

In [12]:
import torch
import torch.nn as nn

class LogCoshLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, y_pred, y_true):
        loss = torch.log(torch.cosh(y_pred - y_true + 1e-12))  # added epsilon to prevent log(0)
        return torch.mean(loss)


In [13]:
import copy

def train_model(model, dataloader, val_dataloader=None, epochs=10, lr=1e-3, patience=10, min_delta=1e-4):
    criterion = LogCoshLoss()
    # criterion = nn.SmoothL1Loss()
    # criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    loss_history = []
    val_loss_history = []

    best_val_loss = float('inf')
    best_model_state = copy.deepcopy(model.state_dict())  # <- to store best weights
    epochs_no_improve = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for xb, yb in dataloader:
            pred = model(xb)
            loss = criterion(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        loss_history.append(avg_loss)

        if val_dataloader:
            model.eval()
            val_total_loss = 0
            with torch.no_grad():
                for xb, yb in val_dataloader:
                    pred = model(xb)
                    loss = criterion(pred, yb)
                    val_total_loss += loss.item()
            avg_val_loss = val_total_loss / len(val_dataloader)
            val_loss_history.append(avg_val_loss)

            if best_val_loss - avg_val_loss > min_delta:
                best_val_loss = avg_val_loss
                best_model_state = copy.deepcopy(model.state_dict())  # save best model
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= patience:
                break
        else:
            print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    if val_dataloader:
        model.load_state_dict(best_model_state)  # <- restore best model weights

    return loss_history, val_loss_history if val_dataloader else loss_history


In [14]:
def plot_loss(loss_history):
    plt.figure(figsize=(10, 4))
    plt.plot(loss_history, label='Training loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'Loss per Epoch')
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

In [15]:
def plot_shap_importance(importances, feature_names=None):
    import matplotlib.pyplot as plt
    import numpy as np

    indices = np.argsort(importances)[::-1]
    names = list(feature_names) if feature_names is not None else [f"Feature {i}" for i in range(len(importances))]

    plt.figure(figsize=(12, 6))
    plt.bar(range(len(importances)), importances[indices])
    plt.xticks(range(len(importances)), [names[i] for i in indices], rotation=45, ha='right')
    plt.title("SHAP Feature Importance (Averaged)")
    plt.tight_layout()
    plt.show()


In [16]:
def predict_from_datetime(model, df, timestamp, n_steps=48, target_col='Price (USD/MWh)', scaler_y=None):
    """
    Predict 24-hour prices starting from a given timestamp.

    Args:
        model: Trained LSTMForecast model.
        df: Preprocessed + scaled DataFrame (with DateTime index).
        timestamp: Datetime string or pd.Timestamp (e.g. '2023-01-01 00:00').
        n_steps: Number of past hours to use (default = 48).
        target_col: Name of target column.
        scaler_y: Scaler used for the target column.

    Returns:
        List of (datetime, predicted_price) tuples.
    """
    if isinstance(timestamp, str):
        timestamp = pd.Timestamp(timestamp)
        
    # Check if enough history is available
    start_idx = df.index.get_loc(timestamp)
    if start_idx < n_steps:
        raise ValueError("Not enough history before this timestamp.")

    # Build the input sequence (excluding target columns)
    seq_df = df.iloc[start_idx - n_steps:start_idx].drop(columns=[target_col, 'target_scaled'])
    seq_input = seq_df.values  # shape: (n_steps, num_features)

    # Predict the future prices
    model.eval()
    with torch.no_grad():
        x = torch.tensor(seq_input[np.newaxis, :, :], dtype=torch.float32)
        y_pred = model(x).squeeze().numpy()  # shape: (forecast_horizon, )

    # Reverse scaling (if provided)
    if scaler_y is not None:
        y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1))  # Inverse transform predictions
        y_pred_original_flat = y_pred_original.flatten()  # Flatten to 1D
    else:
        y_pred_original_flat = y_pred  # If no scaler, use raw predictions

    # Build future timestamps
    future_times = [timestamp + pd.Timedelta(hours=i) for i in range(24)]
    
    return list(zip(future_times, y_pred_original_flat))


# Training model

In [17]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Preprocess (make sure 'Date' becomes index)
df_clean = preprocess(df)

# Define target
target_col = 'Price (USD/MWh)'
features = df_clean.drop(columns=[target_col])
target = df_clean[target_col]

# Columns by type
minmax_cols = ['Tati- normalised output', 'E_Grid (Mw)', 'Revenues (USD)', 
               'Flow_chavuma', 'Level_kariba', 'Flow_nana']
standard_cols = ['Volatility_1 Day', 'Volatility_3 Days', 'Volatility_7 Days', 'Volatility_30 Days',
                 'roc_49h', 'momentum_49h']
no_scaling_cols = ['Hour_sin', 'Hour_cos', 'Month_sin', 'Month_cos',
                   'day_of_week_sin', 'day_of_week_cos']

# Initialize scalers
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()
scaler_y = MinMaxScaler()

# Copy clean DataFrame
df_scaled = df_clean.copy()

# Apply scalers to appropriate columns
df_scaled[minmax_cols] = minmax_scaler.fit_transform(df_clean[minmax_cols])
df_scaled[standard_cols] = standard_scaler.fit_transform(df_clean[standard_cols])

# Target scaling (fit only on the column, keep shape)
df_scaled["target_scaled"] = scaler_y.fit_transform(df_clean[[target_col]])

# Optionally retain unscaled target for reference
df_scaled[target_col] = target


/var/folders/t_/0kjgy0zn0k10thg9kn806w140000gn/T/ipykernel_48089/4031471906.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').dropna()


In [18]:
lookback = 24 * 7 # 7 days of history
# Create sequences
X, y = create_sequences(df_scaled.drop(columns=['Price (USD/MWh)']), target_col='target_scaled', n_steps=lookback, forecast_horizon=24)

# Train-validation-test split (70-15-15 split)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

# Create datasets
train_ds = PriceDataset(X_train, y_train)
val_ds = PriceDataset(X_val, y_val)
test_ds = PriceDataset(X_test, y_test)

# Create DataLoaders
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)

In [ ]:
import optuna
from torch.utils.data import Subset, DataLoader

# Define input size based on your data
input_size = X.shape[2]  # Number of features (should match the input size of the LSTM)

def objective(trial):
    # Suggest hyperparameters
    hidden_size = trial.suggest_int('hidden_size', 32, 96)
    num_layers = trial.suggest_int('num_layers', 1, 2)
    dropout = trial.suggest_float('dropout', 0.1, 0.25)
    lr = trial.suggest_float('lr', 5e-4, 1e-3, log=True)

    # Use subset of training data for faster tuning (1 year - lookback)
    N_subset = 8760 - lookback
    subset_indices = list(range(N_subset))
    subset_train_ds = Subset(train_ds, subset_indices)
    subset_train_loader = DataLoader(subset_train_ds, batch_size=64, shuffle=True)

    # Create model
    model = LSTMForecast(
        input_size=input_size,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout=dropout,
        output_size=24
    ).to(device)

    # Train and evaluate
    _, val_losses = train_model(
        model,
        dataloader=subset_train_loader,
        val_dataloader=val_loader,
        epochs=100,  # Keep same as WOA
        lr=lr,
        patience=15,
        min_delta=1e-4
    )

    return min(val_losses)  # Lower is better

# Create study and run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)  # You can increase n_trials as needed

# Best results
print("✅ Best hyperparameters from Optuna:")
print(study.best_params)
print(f"📉 Best Validation Loss: {study.best_value:.6f}")


In [ ]:
import optuna.visualization as vis

# Optimization history plot (objective value vs. trial number)
vis.plot_optimization_history(study).show()

# Parameter importance plot (shows which hyperparameters mattered most)
vis.plot_param_importances(study).show()

# Parallel coordinates plot (visualizes relationships between parameters and the objective)
vis.plot_parallel_coordinate(study).show()

# Slice plot (performance variation by each hyperparameter)
vis.plot_slice(study).show()

# Model Evaluation 

In [23]:
# Get predictions from model
model.eval()  # Switches the model to eval mode
with torch.no_grad():
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_val_pred = model(X_val_tensor)
    y_test_pred = model(X_test_tensor)

# Reverse scaling for the 24-hour forecast predictions (after prediction)
y_val_pred_original = scaler_y.inverse_transform(y_val_pred.detach().numpy())  # Inverse transform directly
y_test_pred_original = scaler_y.inverse_transform(y_test_pred.detach().numpy())  # Inverse transform directly

# Flatten the predictions to 1D
y_val_pred_original_flat = y_val_pred_original.flatten()  # Flatten to 1D
y_test_pred_original_flat = y_test_pred_original.flatten()  # Flatten to 1D

# Reverse scaling of actual values (if needed)
y_val_original = scaler_y.inverse_transform(y_val.reshape(-1, 1))  # Reverse scaling for true values
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1))  # Reverse scaling for true values

# Flatten the true values
y_val_flat = y_val_original.flatten()
y_test_flat = y_test_original.flatten()


In [24]:
# Create datetime index from original DataFrame
start_date = df.index.min()
end_date = df.index.max()
all_datetimes = pd.date_range(start=start_date, end=end_date, freq='h')
N_total = len(all_datetimes)

# Recalculate split indices
train_size = int(0.7 * N_total)
val_size = int(0.15 * N_total)
val_start = train_size
val_end = train_size + val_size
test_start = val_end

In [25]:
# Generate datetime index for predictions
val_index_expanded = pd.date_range(start=all_datetimes[val_start], periods=len(y_val_flat), freq='h')
test_index_expanded = pd.date_range(start=all_datetimes[test_start], periods=len(y_test_flat), freq='h')

# Create DataFrames for evaluation
X_val_df = pd.DataFrame(index=val_index_expanded)
X_test_df = pd.DataFrame(index=test_index_expanded)


In [26]:
import plotly.graph_objects as go

def plot_predictions(
    y_true, y_pred, df_index,
    start_time="2023-08-01 00:00:00",
    n_hours=500,
    error_threshold=15
):
    # Convert inputs
    start_time = pd.to_datetime(start_time)

    # Build aligned DataFrame
    df = pd.DataFrame({
        'actual': y_true,
        'predicted': y_pred
    }, index=pd.to_datetime(df_index))

    # Slice the time range
    df_slice = df.loc[start_time : start_time + pd.Timedelta(hours=n_hours)]

    # Calculate error
    df_slice['error'] = abs(df_slice['actual'] - df_slice['predicted'])

    # Identify high error regions
    high_error_mask = df_slice['error'] > error_threshold

    # Create Plotly figure
    fig = go.Figure()

    # Actual values (Deep blue)
    fig.add_trace(go.Scatter(
        x=df_slice.index, y=df_slice['actual'],
        mode='lines', name='True',
        line=dict(color='#1f77b4', width=2)
    ))

    # Predicted values (Soft green)
    fig.add_trace(go.Scatter(
        x=df_slice.index, y=df_slice['predicted'],
        mode='lines', name='Predicted',
        line=dict(color='#2ca02c', width=2)
    ))

    # High-error markers (Bold red)
    if high_error_mask.any():
        fig.add_trace(go.Scatter(
            x=df_slice.index[high_error_mask],
            y=df_slice['predicted'][high_error_mask],
            mode='markers',
            marker=dict(size=6, color='#d62728', symbol='circle'),
            name=f'Error > {error_threshold}',
            text=[f"Error: {e:.2f}" for e in df_slice['error'][high_error_mask]],  # Hover text
            hoverinfo='text+x+y',
            showlegend=True
        ))

    # Layout styling
    fig.update_layout(
        title=f"LSTM Forecast from {start_time.strftime('%Y-%m-%d %H:%M')} ({n_hours} hours)",
        xaxis_title="Time",
        yaxis_title="Value",
        template="plotly_white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        height=500,
        margin=dict(l=40, r=40, t=60, b=40)
    )

    fig.show()

In [27]:
plot_predictions(
    y_true=y_val_flat,
    y_pred=y_val_pred_original_flat,
    df_index=X_val_df.index,
    start_time="2023-08-01 00:00:00",
    n_hours=500
)

/var/folders/t_/0kjgy0zn0k10thg9kn806w140000gn/T/ipykernel_48089/2804165366.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slice['error'] = abs(df_slice['actual'] - df_slice['predicted'])


In [28]:
# --- Evaluate model ---
results = evaluate_lstm(
    y_val=y_val_flat,
    y_val_pred=y_val_pred_original_flat,
    y_test=y_test_flat,
    y_test_pred=y_test_pred_original_flat,
    X_val=X_val_df,
    X_test=X_test_df)

print(results)

           Metric        MAE        DAE       RMSE        R2  \
0  Validation Set  32.014972  20.296840  49.865458  0.624994   
1        Test Set  55.602929  49.239382  77.081430  0.204242   

   Lower Predictions (%)  
0              44.895535  
1              76.020319  


In [29]:
def predict_from_datetime(model, df, timestamp, n_steps=48, target_col='Price (USD/MWh)', scaler_y=None):
    """
    Predict 24-hour prices starting from a given timestamp.

    Args:
        model: Trained LSTMForecast model.
        df: Preprocessed + scaled DataFrame (with DateTime index).
        timestamp: Datetime string or pd.Timestamp (e.g. '2023-01-01 00:00').
        n_steps: Number of past hours to use (default = 48).
        target_col: Name of target column.
        scaler_y: Scaler used for the target column.

    Returns:
        List of (datetime, predicted_price) tuples.
    """
    if isinstance(timestamp, str):
        timestamp = pd.Timestamp(timestamp)
        
    # Check if enough history is available
    start_idx = df.index.get_loc(timestamp)
    if start_idx < n_steps:
        raise ValueError("Not enough history before this timestamp.")

    # Build the input sequence (excluding target columns)
    seq_df = df.iloc[start_idx - n_steps:start_idx].drop(columns=[target_col, 'target_scaled'])
    seq_input = seq_df.values  # shape: (n_steps, num_features)

    # Predict the future prices
    model.eval()
    with torch.no_grad():
        x = torch.tensor(seq_input[np.newaxis, :, :], dtype=torch.float32)
        y_pred = model(x).squeeze().numpy()  # shape: (forecast_horizon, )

    # Reverse scaling (if provided)
    if scaler_y is not None:
        y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1))  # Inverse transform predictions
        y_pred_original_flat = y_pred_original.flatten()  # Flatten to 1D
    else:
        y_pred_original_flat = y_pred  # If no scaler, use raw predictions

    # Build future timestamps
    future_times = [timestamp + pd.Timedelta(hours=i) for i in range(24)]
    
    return list(zip(future_times, y_pred_original_flat))


In [225]:
preds = predict_from_datetime(model, df_scaled, '2024-01-01 00:00',scaler_y=scaler_y)

for t, price in preds:
    print(f"{t}: ${price:.2f}")

2024-01-01 00:00:00: $36.86
2024-01-01 01:00:00: $33.13
2024-01-01 02:00:00: $37.40
2024-01-01 03:00:00: $36.03
2024-01-01 04:00:00: $37.60
2024-01-01 05:00:00: $46.29
2024-01-01 06:00:00: $41.06
2024-01-01 07:00:00: $35.85
2024-01-01 08:00:00: $39.13
2024-01-01 09:00:00: $41.88
2024-01-01 10:00:00: $51.75
2024-01-01 11:00:00: $40.98
2024-01-01 12:00:00: $41.39
2024-01-01 13:00:00: $49.47
2024-01-01 14:00:00: $50.68
2024-01-01 15:00:00: $42.93
2024-01-01 16:00:00: $32.74
2024-01-01 17:00:00: $56.77
2024-01-01 18:00:00: $132.42
2024-01-01 19:00:00: $178.42
2024-01-01 20:00:00: $172.41
2024-01-01 21:00:00: $105.71
2024-01-01 22:00:00: $56.15
2024-01-01 23:00:00: $44.99


#  Post-Hoc Interpretability with Integrated Gradients (XAI)
To understand how the LSTM model arrives at its predictions, we apply post-hoc interpretability methods—techniques used after model training to explain behavior without altering the model itself. One such method is Integrated Gradients, which attributes importance scores to input features based on their contribution to a specific prediction. This approach is particularly valuable for complex, black-box models like LSTMs, where internal mechanisms are not easily interpretable. By applying Integrated Gradients, we gain insights into both individual decisions (local interpretation) and broader model behavior (global interpretation), helping to build transparency, trust, and accountability in the forecasting process.


## Local Interpretation
The local interpretation using Integrated Gradients provides insight into which input features most influenced the model’s prediction for a specific timestamp. By attributing importance values to each feature in that one sample, we can understand the direction (positive or negative) and magnitude of their impact on the predicted price. This helps explain the model's reasoning at an individual decision level — for example, highlighting that high 7-day volatility or weekend timing pushed the forecast upward in that particular context.

In [ ]:
from captum.attr import IntegratedGradients
import torch
import matplotlib.pyplot as plt

# Ensure model is in eval mode
model.eval()

# Extract feature names from your DataFrame
feature_names = features.columns.tolist()  # must match order in X_val

# Select a sample
sample_idx = 0 # sample_idx is simply the index of a single sample (row) in your validation dataset (X_val)
input_tensor = torch.tensor(X_val[sample_idx:sample_idx+1], dtype=torch.float32, requires_grad=True)

# Initialize Integrated Gradients
ig = IntegratedGradients(model)

# Compute attributions for the first output (e.g., first forecasted hour)
attributions, delta = ig.attribute(
    input_tensor, target=0, return_convergence_delta=True
)

# Sum attributions across the time dimension
attributions_sum = attributions.sum(dim=1).squeeze().detach().numpy()

# Plot feature attributions with proper labels
plt.figure(figsize=(12, 6))
plt.bar(feature_names, attributions_sum)
plt.xticks(rotation=45, ha='right')
plt.xlabel("Feature")
plt.ylabel("Attribution")
plt.title("Local Feature Attributions using Integrated Gradients")
plt.tight_layout()
plt.show()

The global interpretation aggregates feature attributions across many samples to show which inputs the model relies on most consistently. By averaging importance scores, it highlights the overall influence of each feature on predictions—revealing patterns such as persistent reliance on long-term volatility or cyclical time features. This offers a high-level understanding of the model’s behavior and helps validate that it aligns with domain knowledge.

In [ ]:
all_attributions = []
model.eval()

for i in range(len(X_val)):
    input_tensor = torch.tensor(X_val[i:i+1], dtype=torch.float32, requires_grad=True)
    attributions, _ = ig.attribute(input_tensor, target=0, return_convergence_delta=True)
    summed = attributions.sum(dim=1).squeeze().detach().numpy()
    all_attributions.append(summed)

# Now average across all samples
avg_attr = np.mean(np.stack(all_attributions), axis=0)

# Plot global feature importance
plt.figure(figsize=(12, 6))
plt.bar(feature_names, avg_attr)
plt.xticks(rotation=45, ha='right')
plt.ylabel("Average Attribution")
plt.title("Global Feature Importance via Integrated Gradients")
plt.tight_layout()
plt.show()

